In [33]:
import re
import pandas as pd

https://stackoverflow.com/questions/79042960/regex-to-differentiate-chemical-compound-names-from-pattern

In [20]:
tokens = [
    "Compound Name,RT,Score\n2,4,6-Cycloheptatrien-1-one, 3,5-bis-trimethylsilyl-,8.705,626,Area,386902.453,tR (min),8.705",
    "Compound Name,RT,Score\nBenzene, 4-ethenyl-1,2-dimethyl-,11.682,600,Area,16289.239,tR (min),11.682",
    "Compound Name,RT,Score\n2-(p-Tolyl)propan-2-ol, TMS,11.921,605,Area,17976.558,tR (min),11.921",
    "Compound Name,RT,Score\nTris(tert-butyldimethylsilyloxy)arsane,12.036,685,Area,76702.743,tR (min),12.036",
    "Compound Name,RT,Score\nPhenyl-pentamethyl-disiloxane,13.646,656,Area,84633.281,tR (min),13.646",
    "Compound Name,RT,Score\n**[1,1'-Bicyclopropyl]-2-octanoic acid, 2'-hexyl-, methyl est**,16.642,605,Area,566.512,tR (min),16.642",
    "Compound Name,RT,Score\n**(t-Butyl-dimethylsilyl)[2-methyl-2-(4-methyl-pent-3-enyl)-c**,19.853,650,Area,10141.071,tR (min),19.853",
]

In [23]:
def process(token):
    lines = token.split("\n")
    fields = lines[-1].split(",")
    return {
        "compound": ",".join(fields[:-6]),
        "area": fields[-3],
        "rt": fields[-1],
    }

In [27]:
results = pd.DataFrame([process(token) for token in tokens])
print(results)

                                            compound        area      rt
0  2,4,6-Cycloheptatrien-1-one, 3,5-bis-trimethyl...  386902.453   8.705
1                   Benzene, 4-ethenyl-1,2-dimethyl-   16289.239  11.682
2                        2-(p-Tolyl)propan-2-ol, TMS   17976.558  11.921
3             Tris(tert-butyldimethylsilyloxy)arsane   76702.743  12.036
4                      Phenyl-pentamethyl-disiloxane   84633.281  13.646
5  **[1,1'-Bicyclopropyl]-2-octanoic acid, 2'-hex...     566.512  16.642
6  **(t-Butyl-dimethylsilyl)[2-methyl-2-(4-methyl...   10141.071  19.853


In [28]:
sample = """Sequence Name,20240603_S24031_anonymous
Sample Name,Blank
Area,15906292.428,tR (min),3.696
Area,10164232.248,tR (min),3.783
Area,5541647.716,tR (min),4.091
Area,203969.547,tR (min),6.514
Area,542692.442,tR (min),6.579
Area,1039445.806,tR (min),6.788
Area,4011338.408,tR (min),7.249
Area,504687.529,tR (min),8.177
Area,108363.994,tR (min),8.420
Compound Name,RT,Score
2,4,6-Cycloheptatrien-1-one, 3,5-bis-trimethylsilyl-,8.705,626,Area,386902.453,tR (min),8.705
Area,368393.389,tR (min),8.850
Area,91574.405,tR (min),9.447
Area,899294.259,tR (min),9.651
Area,193389.843,tR (min),10.041
Area,56853.148,tR (min),10.812
Compound Name,RT,Score
Benzene, 4-ethenyl-1,2-dimethyl-,11.682,600,Area,16289.239,tR (min),11.682
Compound Name,RT,Score
2-(p-Tolyl)propan-2-ol, TMS,11.921,605,Area,17976.558,tR (min),11.921
Area,78966.216,tR (min),12.031
Compound Name,RT,Score
Tris(tert-butyldimethylsilyloxy)arsane,12.036,685,Area,76702.743,tR (min),12.036
Area,534.128,tR (min),12.285
Compound Name,RT,Score
Phenyl-pentamethyl-disiloxane,13.646,656,Area,84633.281,tR (min),13.646
Area,11464.379,tR (min),13.696
Compound Name,RT,Score
Cyclopentene, 3,3-dimethyl-4-methylene-1,2-bis(trimethylsil,14.543,645,Area,402308.085,tR (min),14.543
"""

In [48]:
def process(filename):
    with open(filename, "r") as file:
        raw = file.read()
    results = []
    for sequence_id, sequence in enumerate(raw.split("\n\n")):
        lines = sequence.split("\n")
        sequence_name = lines[0].split(",")[-1]
        for line in lines:
            fields = line.split(",")
            data = {}
            if "Area," in line:
                data |= {"area": float(fields[-3]), "rt": float(fields[-1])}
            if ",Area," in line:
                data |= {"compound": ",".join(fields[:-6])}
            if data:
                results.append({
                    "sequence_id": sequence_id,
                    "sequence_name": sequence_name,
                } | data)
    return pd.DataFrame(results)

In [50]:
print(process("agilent_report.csv"))

      sequence_id                sequence_name          area      rt  \
0               0    20240603_S24031_anonymous  1.590629e+07   3.696   
1               0    20240603_S24031_anonymous  1.016423e+07   3.783   
2               0    20240603_S24031_anonymous  5.541648e+06   4.091   
3               0    20240603_S24031_anonymous  2.039695e+05   6.514   
4               0    20240603_S24031_anonymous  5.426924e+05   6.579   
...           ...                          ...           ...     ...   
1675           37  20240604_S24032_BelénFlórez  2.436482e+07  19.036   
1676           37  20240604_S24032_BelénFlórez  5.640226e+06  21.963   
1677           37  20240604_S24032_BelénFlórez  5.021575e+05  23.108   
1678           37  20240604_S24032_BelénFlórez  1.940947e+06  23.592   
1679           37  20240604_S24032_BelénFlórez  9.665506e+05  24.550   

                                  compound  
0                                      NaN  
1                                      NaN  
